In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchtext
from torchtext.data.utils import get_tokenizer

In [2]:
torchtext.datasets.WikiText2

torchtext.datasets.language_modeling.WikiText2

In [3]:
from typing import List
import datetime as dt
import re
import os
from collections import defaultdict
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer
from enum import Enum
from random import choices
from random import randint
import json

mystem = Mystem()

import nltk
nltk.download("stopwords")
#--------#

from nltk.corpus import stopwords
english_stopwords = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/annikura/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
with open('data_jokes_clean.json', 'rt') as file:
    json_jokes = json.load(file)['posts']

with open('train.txt', 'w') as train:
    with open('test.txt', 'w') as test:
        with open('valid.txt', 'w') as valid:
            for joke in json_jokes:
                r = randint(0, 100)
                text = joke['title'] + " " + joke['text']
                text = ' '.join(' '.join(text.split(r'\n')).split()) + '\n'
                if len(text) > 1000 or joke["tag"] == "long" or 'htt' in text or 'edit:' in text or '&#' in text:
                    continue
                if r < 95:
                    train.write(text)
                elif r < 97:
                    valid.write(text)
                else:
                    test.write(text)

Why do black guy's eyes get red after sex. mace

Why do turkeys always gobble? Why do turkeys always gobble? They haven’t been taught good table manners!

Why did the cellphone go to court? Because it was charged with battery. Thank you, I'll be here all night... Finals week needs to end.

I don't say "YO MAMA" jokes anymore... Cause they're all old and used up, just like your mom

The joke is on you... SWEDEN! Here's a few jokes: Q: How do you sink a Swedish submarine? A: You dive and knock on the hatch. Q: Why there is alway a man and a woman in Swedish police patrol? A: To avoid workplace romances. Q: What is a party game played by Swedes? A: One Swede goes into a box and the other Swede tries to guess which Swede is in it. Q: How do you sink a Swedish battleship? A: Let them build it themselves. Q: Why Swedes are bad fishermen? A: When they catch a fish, the try to kill it by drowning.

What did one snowman say to the other snowman? Do you smell carrots?

Anyone have a good birthda

The Lord of an 19th century English manor is having an affair with one of his chambermaids... One day the chambermaid is giving him a blow-job when she hears the lady of the house approaching. She stops what she's doing and looks up at him. At that moment, he climaxes and manages to get some right in her eye. It's messy and burning, she runs to the door, rubbing her face and tearing up. Just as she closes the door to the bedroom, the wife comes around the corner and sees her. The lady asks "What are you doing outside our bedroom!? ...are you crying? What happened my dear?" The girl looks at her and says "I just, I overheard your husband singing a hymn and I got overwhelmed with the spirit I think." "Why that's beautiful. Which hymn?" The chambermaid thinks for a second before blurting out "Battle Hymn of the Republic." "Oh, I'm not familiar. How does it go?" The girl takes a few seconds, wiping her face before singing. "Mine eyes have seen the glory of the coming of the lord..."

Pavlo

My plan if there is a draft: Put a coat on.

I found this place online that sells authentic moon rocks. The rocks themselves are really cheap, but the shipping is a bitch.

I always thought my english grades were good But then in 6th grade, i learned the alphabet

I almost bought a huge library out of old computer programming books... ...but the ascii price was way too high.

I got raped by a troupe of mimes last night They performed unspeakable acts.

What is.. What's black and sits at the top of a staircase? Stephen Hawkins after a house fire.

What kind of karate do Rabbis know? Jew Jitsu.

I went to a blind fortune teller the other day She looked into her crystal ball, and she told me there is eternal darkness in my future.

What is the similarity between poop and aliens? They both live on Uranus

What do you get when you cross a ghoul and a vampire? A hemogoblin. I came up with this during lecture after a dyslexic moment, thought someone may like it.

The CEO of Apple came out gay

How do you keep redditors in suspense? I dunno, I'll tell ya tomorrow

How much money does a gay pornstar earn? A buttload

Yo mama so fat that black holes revolve around her.

Just ask Tommy Hilfiger it out.

What do you call a dog with no legs? It doesn't matter. He won't come anyway.

TIL Supreme Court judge kidnapped by cannibals in the Caribbean! Justice was served.

The gift "Brian, what's wrong with you? You've been sitting at your desk looking depressed for the last five minutes!" "Oh Dan", responded Brian "I don't know what to do! I got in a big fight last night with my Suzie, she claims I never buy her gifts, so I must not care about her!" "Brian, that's horrible!" said Dan putting his arm around Brian. "What type of a gift does she want already?" "Well, right before she closed the door on my face she said to me: Why don't you show me how much you care about me? Why can't you get me something that goes from 0 to 175 in seconds!" "Dan what should I do? I don't have that kind o

In [4]:
def process_to_tokens(text):
    text = (' '.join(text.split(r'\n'))).lower()
    
    while True:
        new_s = re.sub("\\.\\.+", ".", text)
        if new_s == text:
            break
        text = new_s
    
    word = ''
    number = ''
    
    tokens = []
    
    is_whitespace = re.compile('\s')
    for sym in text:
        if sym.isalpha():
            if number:
                tokens.append(number)
                number = ''
            word += sym
            continue
        if sym.isnumeric():
            if word:
                tokens.append(word)
                word = ''
            number += sym
            continue
        if number:
                tokens.append(number)
        if word:
                tokens.append(word)
        number, word = '', ''
        if is_whitespace.match(sym) or sym == r'\\' or sym == "'":
            continue
        tokens.append(sym)
    if number:
                tokens.append(number)
    if word:
            tokens.append(word)
    return tokens